In [ ]:
!pip install ase
!pip install mace-torch
!pip install orb_models
!pip install pynanoflann



In [ ]:
# Pour décompresser les contenus des fichiers `mols.zip` et `solids.zip`, uniquement dans Google Colab. Décompressez manuellement sur un système local.

import os

# Créer les sous-répertoires s'ils n'existent pas.
if not os.path.exists("mols"):
  os.makedirs("mols")
if not os.path.exists("solids"):
  os.makedirs("solids")

!unzip mols.zip -d mols
!unzip solids.zip -d solids

In [ ]:
import sys
import time
from ase.calculators.calculator import Calculator as ASECalculator
from ase.io import read
from ase.filters import FrechetCellFilter
from ase.optimize import QuasiNewton, FIRE
from mace.calculators import mace_mp, mace_anicc
from orb_models.forcefield import pretrained
from orb_models.forcefield.calculator import ORBCalculator


In [ ]:
def compute_x23_case(
    mol_file: str,
    solid_file: str,
    calc: ASECalculator,
    num_mols_per_cell: int,
    f_cutoff: float=0.05,
    max_steps: int=200,
    opt_method: str="qn",
) -> tuple[float, float]:
    """
    Calcule un cas X23 donné et renvoie l'énergie réticulaire + le volume de la cellule.

    :param mol_file: Chemin vers le fichier de la molécule.
    :param solid_file: Chemin vers le fichier du solide.
    :param calc: L'outil de calcul ASE.
    :param num_mols_per_cell: Nombre de molécules dans `solid_file`.
    :param f_cutoff: Seuil des forces pour la convergence, en eV/A.
    :param max_steps: Nombre maximum d'étapes d'optimisation.
    :param opt_method: Méthode d'optimisation ("qn" ou "fire").
    """
    mol_atoms = read(mol_file)
    mol_atoms.calc = calc

    if opt_method == "qn":
        mol_qn = QuasiNewton(mol_atoms)
    elif opt_method == "fire":
        mol_qn = FIRE(mol_atoms)
    else:
        raise ValueError("unknown opt_method")
    mol_qn.run(fmax=f_cutoff, steps=max_steps)

    solid_atoms = read(solid_file)
    solid_atoms.calc = calc

    if opt_method == "qn":
        solid_qn = QuasiNewton(FrechetCellFilter(solid_atoms))
    elif opt_method == "fire":
        solid_qn = FIRE(FrechetCellFilter(solid_atoms))
    else:
        raise ValueError("unknown opt_method")
    solid_qn.run(fmax=f_cutoff, steps=max_steps)

    sE = solid_atoms.get_potential_energy()
    mE = mol_atoms.get_potential_energy()
    lattice_energy = sE / num_mols_per_cell - mE

    opt_volume = solid_atoms.get_volume() / num_mols_per_cell

    return float(lattice_energy) * 23.0605419, float(opt_volume)

In [ ]:
calc = mace_mp()

num_per_cell = [2, 4, 2, 4, 2, 4, 4, 8, 4, 2, 4, 1, 4, 2, 4, 2, 2, 8, 2, 6, 6, 4, 2]

for idx, num in enumerate(num_per_cell):
    i = idx + 1
    start_time = time.time()

    E, vol = compute_x23_case(
        f"x23/mols/{i:02d}.extxyz",
        f"x23/solids/{i:02d}.extxyz",
        calc,
        num,
    )

    end_time = time.time()
    elapsed = end_time - start_time

    with open("mace_mp_x23.out", "a+") as f:
        f.write(f"{i},{E:.5f},{vol:.5f},{elapsed:.1f}\n")

In [ ]:
orbff = pretrained.orb_d3_v2()
calc = ORBCalculator(orbff)

num_per_cell = [2, 4, 2, 4, 2, 4, 4, 8, 4, 2, 4, 1, 4, 2, 4, 2, 2, 8, 2, 6, 6, 4, 2]

for idx, num in enumerate(num_per_cell):
    i = idx + 1
    start_time = time.time()

    E, vol = compute_x23_case(
        f"x23/mols/{i:02d}.extxyz",
        f"x23/solids/{i:02d}.extxyz",
        calc,
        num,
        opt_method="fire",
    )

    end_time = time.time()
    elapsed = end_time - start_time

    with open("orbd3v2_x23.out", "a+") as f:
        f.write(f"{i},{E:.5f},{vol:.5f},{elapsed:.1f}\n")